In [1]:
import sys
import os
sys.path.append("../")

import pandas as pd
import numpy as np
import pickle
from sklearn.cluster import KMeans
from sklearn.metrics import  silhouette_score, silhouette_samples
import matplotlib.pyplot as plt
from IPython.display import HTML
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy import stats

## Reading the change object and clustering.

In [2]:
article_name = "Truth"
change_object_dir =  "../data/change objects/"
change_vector_dir = "../data/change_vector/"

change_object_file_name = f"{article_name}_vec.npz"
filename =  f"{article_name}_change.h5"

change_object_file = os.path.join(change_object_dir, filename)
change_vector_file = os.path.join(change_vector_dir, change_object_file_name)

In [3]:
%%time
if os.path.exists(change_object_file):
    with pd.HDFStore(change_object_file, 'r') as store:
        change_object_dataframe = store.get("data")
else:
    print("file do not exist")

CPU times: user 1.25 s, sys: 19.9 s, total: 21.2 s
Wall time: 21.1 s


In [4]:
change_object_dataframe.head()

ins_tokens  \
from revision id to revision id timestamp           timegap          editor                                                                
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0                                       (-, -, -, -)   
                                                                                    1                                       (foregoing,)   
                                                                                    2  ([[, larrys, text, ]], ., if, you, can, do, be...   
                                                                                    3  (you, can, feel, free, to, radically, update, ...   
                                                                                    4                                                 ()   

                                                                                         del_tokens  \
from revision id to revision id timestamp           timegap          editor                           
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0            ()   
                                                                                    1            ()   
                                                                                    2            ()   
                                                                                    3            ()   
                                                                                    4  (following,)   

                                                                                                    left_neigh  \
from revision id to revision id timestamp           timegap          editor                                      
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0        slice(0, 3, None)   
                                                                                    1        slice(0, 8, None)   
                                                                                    2       slice(0, 10, None)   
                                                                                    3  slice(4212, 4243, None)   
                                                                                    4        slice(0, 3, None)   

                                                                                               right_neigh  \
from revision id to revision id timestamp           timegap          editor                                  
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0   slice(1, 32, None)   
                                                                                    1   slice(4, 35, None)   
                                                                                    2   slice(9, 40, None)   
                                                                                    3  slice(13, 44, None)   
                                                                                    4   slice(4, 35, None)   

                                                                                                                              left_token  \
from revision id to revision id timestamp           timegap          editor                                                                
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0                                  ({st@rt}, <, the)   
                                                                                    1   ({st@rt}, <, the, following, is, a, portion, of)   
                                                                                    2  ({st@rt}, <, the, following, is, a, portion, o...   
                                                                                    3  (to, rest, content, with, any, sort, of, relat...   
             

## Read Vectors of change object.

In [5]:
with open(change_vector_file, "rb") as file:
    arrays_dict = np.load(file)
    neighbour_10_matrix = arrays_dict["neighbour_10"]
    ins_del_10_sum_neighbour_matrix = arrays_dict["ins_del_10_sum_neighbour"]
    neighbour_4_matrix = arrays_dict["neighbour_4"]
    ins_del_4_sum_neighbour_matrix = arrays_dict["ins_del_4_sum_neighbour"]
    weighted_neighbour_matrix_matrix = arrays_dict["weighted_neighbour_matrix"]
    ins_del_weighted_neighbour_matrix = arrays_dict["ins_del_weighted_neighbour_matrix"]

### Make left, ins and delete string for visualisation

In [6]:
change_object_dataframe["left_string"] = change_object_dataframe["left_token"].str.join(" ")
change_object_dataframe["ins_string"] = change_object_dataframe["ins_tokens"].str.join(" ")
change_object_dataframe["del_string"] = change_object_dataframe["del_tokens"].str.join(" ")
change_object_dataframe["right_string"] = change_object_dataframe["right_token"].str.join(" ")

In [7]:
change_object_dataframe["ins_token_len"]=change_object_dataframe["ins_tokens"].str.len()
change_object_dataframe["del_token_len"]=change_object_dataframe["del_tokens"].str.len()

## Clustering

In [8]:
style_dict = {'border': "2px solid #000",
              "text-align": "justify"
    
}

In [9]:
%%time
NO_OF_CLUSTERS = 70
km = KMeans(n_clusters= NO_OF_CLUSTERS, n_jobs=3)
clusters_4 = km.fit(neighbour_4_matrix)

CPU times: user 7min 45s, sys: 38.3 s, total: 8min 23s
Wall time: 2min 8s


/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (70). Possibly due to duplicate points in X.
  return_n_iter=True)


In [10]:
change_object_dataframe["cluster_4"] = pd.Series(clusters_4.labels_, index= change_object_dataframe.index)
change_grouped_by_tokens_4_neigh = change_object_dataframe.groupby("cluster_4")

In [11]:
repers_4_neigh = [ change_grouped_by_tokens_4_neigh[["left_string","del_string", "ins_string", "right_string"]].get_group(i).style.set_caption("group "+ str(i) ).render() for i in range(NO_OF_CLUSTERS)]

KeyError: 1

In [ ]:
all_html = " ".join(repers_4_neigh)
file_name = article_name + "_4_neigh_"+str(NO_OF_CLUSTERS) + "_clusters.html"
file_path = os.path.join("./visualisation", file_name)
with open(file_path, 'wb') as f:
    f.write(all_html.encode())

In [ ]:
@interact( clusters_html=fixed(repers_4_neigh), group=range(70))
def display_clusters(clusters_html, group):
     return display(HTML(clusters_html[group]))


##### Cluster number of neighbour tokens=10, number of clusters =100 

In [ ]:
%%time
NO_OF_CLUSTERS = 100
km = KMeans(n_clusters= NO_OF_CLUSTERS, n_jobs=3)
clusters_10 = km.fit(neighbour_10_matrix)

In [ ]:
change_object_dataframe["cluster_10"] = pd.Series(clusters_10.labels_, index= change_object_dataframe.index)
change_grouped_by_tokens_10_neigh = change_object_dataframe.groupby("cluster_10")

In [ ]:
repers_10_neigh = [ change_grouped_by_tokens_10_neigh[["left_string","del_string", "ins_string", "right_string"]].get_group(i).style.set_properties(**style_dict, axis=None).set_caption("group "+ str(i) ).render() for i in range(NO_OF_CLUSTERS)]

In [ ]:
all_html = " ".join(repers_10_neigh)
file_name = article_name + "_10_neigh_"+str(NO_OF_CLUSTERS) +"_clusters.html"
file_path = os.path.join("./visualisation", file_name)
with open(file_path, 'wb') as f:
    f.write(all_html.encode())

In [ ]:
@interact( clusters_html=fixed(repers_10_neigh), group=range(100))
def display_clusters(clusters_html, group):
     return display(HTML(clusters_html[group]))


In [ ]:
# id = list(range(20))
# id_str = ["group "+str(i) for i in id]
# drop_down = list(zip(id_str, id))
# # interact(display_clusters, clusters_html=fixed(repers), group= drop_down);
# interact(display_clusters, clusters_html=fixed(repers),  group= drop_down)


### Clustering with vectors concatinated vectors average of 4 right and left neighbours and average of inserted and deleted tokens. 

In [ ]:
%%time
NO_OF_CLUSTERS = 100
km = KMeans(n_clusters= NO_OF_CLUSTERS, n_jobs=3)
clusters_4_full = km.fit(ins_del_4_sum_neighbour_matrix)
change_object_dataframe["cluster_4_full"] = pd.Series(clusters_4_full.labels_, index= change_object_dataframe.index)

In [ ]:
change_grouped_by_tokens_4_full = change_object_dataframe.groupby("cluster_4_full")

In [ ]:
repers_4_full = [ change_grouped_by_tokens_4_full[["left_string","del_string", "ins_string", "right_string"]].get_group(i).style.set_caption("group "+ str(i) ).render() for i in range(NO_OF_CLUSTERS)]

In [ ]:
all_html = " ".join(repers_4_full)
file_name = article_name + "_4_full_"+str(NO_OF_CLUSTERS) + "_clusters.html"
file_path = os.path.join("./visualisation", file_name)
with open(file_path, 'wb') as f:
    f.write(all_html.encode())

In [ ]:
@interact( clusters_html=fixed(repers_4_full), group=range(100))
def display_clusters(clusters_html, group):
     return display(HTML(clusters_html[group]))


### Saving the cluster with change object
###### TO-DO: save change object and cluster seperately.

In [ ]:
cluster_dir = "../data/clusters/"
file_name = article_name + "_cluster.h5"
full_file_path = os.path.join(cluster_dir, file_name)
with pd.HDFStore(full_file_path, 'w') as store:
    store.put("change_object", change_object_dataframe, table=False)

### Ranking
###### Ranking clustered groups on following parameters.
1. Size of clusters
2. No of unique editors is clusters
3. Total period of cluster. i.e difference between start and end date.
4. Median length of edited token in each cluster.

In [ ]:
rank_by_size = change_grouped_by_tokens.size().sort_values()
rank_by_size

In [ ]:
rank_by_uniq_editor = change_grouped_by_tokens["editor_s"].nunique().sort_values()

In [ ]:
rank_by_period = change_grouped_by_tokens["timestamp"].apply(lambda x: x.max() - x.min()).sort_values()

In [ ]:
rank_by_rate = change_grouped_by_tokens["time_gap"].apply(lambda x: x.mean()).sort_values()

In [ ]:
rank_by_token_length = (change_grouped_by_tokens["ins_token_len"].median() + change_grouped_by_tokens["del_token_len"].median()).sort_values()
rank_by_token_length = ranks_by_token_length /2

In [ ]:
plt.scatter(np.log10(cluster_ranks_by_size), rank_by_uniq_editor)
plt.scatter(np.log10(cluster_ranks_by_size), rank_by_token_length)
plt.scatter(np.log10(cluster_ranks_by_size), np.log10(pd.to_numeric(rank_by_period)))
plt.scatter(np.log10(cluster_ranks_by_size), np.log10(pd.to_numeric(rank_by_rate)))

In [ ]:
change_grouped_by_tokens.get_group(cluster_ranks_by_period.index.tolist()[0])

In [ ]:
change_grouped_by_tokens.get_group(cluster_ranks_by_period.index.tolist()[0])["del_string"].str.cat()

In [ ]:
%%time
silhoutee_s = []
for n in range(2,48,2):
    km = KMeans(n_clusters= n, n_jobs=3)
    clusters = km.fit(ins_del_sum_neighbour_matrix)
    cluster_s = pd.Series(clusters.labels_, index= change_df.index)
    silhoutee_s.append(silhouette_score(change_matrix, cluster_s))

In [ ]:
x = np.arange(2,48,2)
plt.plot(x,silhoutee_s)

In [ ]:
x[np.argsort(silhoutee_s)[-8:]]

In [ ]:
np.array(silhoutee_s)[np.argsort(silhoutee_s)[-8:]]

In [ ]:
%%time
km = KMeans(n_clusters= 12, n_jobs=3)
clusters = km.fit(ins_del_sum_neighbour_matrix)
cluster_s = pd.Series(clusters.labels_, index= change_df.index)
change_df["cluster"] = cluster_s
change_grouped_by_sum = change_df.groupby("cluster")
change_grouped_by_sum.size().plot()

In [ ]:
rank_by_size = change_grouped_by_sum.size().sort_values()
rank_by_size

In [ ]:
rank_by_uniq_editor = change_grouped_by_sum["editor_s"].nunique().sort_values()

In [ ]:
rank_by_period = change_grouped_by_sum["timestamp"].apply(lambda x: x.max() - x.min()).sort_values()

In [ ]:
rank_by_rate = change_grouped_by_sum["time_gap"].apply(lambda x: x.mean()).sort_values()

In [ ]:
rank_by_token_length = (change_grouped_by_sum["ins_token_len"].median() + change_grouped_by_sum["del_token_len"].median()).sort_values()
rank_by_token_length = ranks_by_token_length /2

In [ ]:
plt.scatter(np.log10(cluster_ranks_by_size), rank_by_uniq_editor)
plt.scatter(np.log10(cluster_ranks_by_size), rank_by_token_length)
plt.scatter(np.log10(cluster_ranks_by_size), np.log10(pd.to_numeric(rank_by_period)))
plt.scatter(np.log10(cluster_ranks_by_size), np.log10(pd.to_numeric(rank_by_rate)))

In [ ]:
change_grouped_by_sum.get_group(9)[["del_string", "ins_string"]]